In [24]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

#fmt:off

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 13: Packet Scanners ---</h2><p>You need to cross a vast <em>firewall</em>. The firewall consists of several layers, each with a <em>security scanner</em> that moves back and forth across the layer. To succeed, you must not be detected by a scanner.</p>
<p>By studying the firewall briefly, you are able to record (in your puzzle input) the <em>depth</em> of each layer and the <em>range</em> of the scanning area for the scanner within it, written as <code>depth: range</code>. Each layer has a thickness of exactly <code>1</code>. A layer at depth <code>0</code> begins immediately inside the firewall; a layer at depth <code>1</code> would start immediately after that.</p>
<p>For example, suppose you've recorded the following:</p>
<pre><code>0: 3
1: 2
4: 4
6: 4
</code></pre>
<p>This means that there is a layer immediately inside the firewall (with range <code>3</code>), a second layer immediately after that (with range <code>2</code>), a third layer which begins at depth <code>4</code> (with range <code>4</code>), and a fourth layer which begins at depth 6 (also with range <code>4</code>). Visually, it might look like this:</p>
<pre><code> 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]
</code></pre>
<p>Within each layer, a security scanner moves back and forth within its range. Each security scanner starts at the top and moves down until it reaches the bottom, then moves up until it reaches the top, and repeats. A security scanner takes <em>one picosecond</em> to move one step.  Drawing scanners as <code>S</code>, the first few picoseconds look like this:</p>
<pre><code>
Picosecond 0:
 0   1   2   3   4   5   6
[S] [S] ... ... [S] ... [S]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]

Picosecond 1:
0 1 2 3 4 5 6
[ ] [ ] ... ... [ ] ... [ ]
[S] [S] [S] [S]
[ ] [ ] [ ]
[ ] [ ]

Picosecond 2:
0 1 2 3 4 5 6
[ ] [S] ... ... [ ] ... [ ]
[ ] [ ] [ ] [ ]
[S] [S] [S]
[ ] [ ]

Picosecond 3:
0 1 2 3 4 5 6
[ ] [ ] ... ... [ ] ... [ ]
[S] [S] [ ] [ ]
[ ] [ ] [ ]
[S] [S]
</code></pre>

<p>Your plan is to hitch a ride on a packet about to move through the firewall.  The packet will travel along the top of each layer, and it moves at <em>one layer per picosecond</em>. Each picosecond, the packet moves one layer forward (its first move takes it into layer 0), and then the scanners move one step. If there is a scanner at the top of the layer <em>as your packet enters it</em>, you are <em>caught</em>. (If a scanner moves into the top of its layer while you are there, you are <em>not</em> caught: it doesn't have time to notice you before you leave.) If you were to do this in the configuration above, marking your current position with parentheses, your passage through the firewall would look like this:</p>
<pre><code>Initial state:
 0   1   2   3   4   5   6
[S] [S] ... ... [S] ... [S]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]

Picosecond 0:
0 1 2 3 4 5 6
(S) [S] ... ... [S] ... [S]
[ ] [ ] [ ] [ ]
[ ] [ ] [ ]
[ ] [ ]

0 1 2 3 4 5 6
( ) [ ] ... ... [ ] ... [ ]
[S] [S] [S] [S]
[ ] [ ] [ ]
[ ] [ ]

Picosecond 1:
0 1 2 3 4 5 6
[ ] ( ) ... ... [ ] ... [ ]
[S] [S] [S] [S]
[ ] [ ] [ ]
[ ] [ ]

0 1 2 3 4 5 6
[ ] (S) ... ... [ ] ... [ ]
[ ] [ ] [ ] [ ]
[S] [S] [S]
[ ] [ ]

Picosecond 2:
0 1 2 3 4 5 6
[ ] [S] (.) ... [ ] ... [ ]
[ ] [ ] [ ] [ ]
[S] [S] [S]
[ ] [ ]

0 1 2 3 4 5 6
[ ] [ ] (.) ... [ ] ... [ ]
[S] [S] [ ] [ ]
[ ] [ ] [ ]
[S] [S]

Picosecond 3:
0 1 2 3 4 5 6
[ ] [ ] ... (.) [ ] ... [ ]
[S] [S] [ ] [ ]
[ ] [ ] [ ]
[S] [S]

0 1 2 3 4 5 6
[S] [S] ... (.) [ ] ... [ ]
[ ] [ ] [ ] [ ]
[ ] [S] [S]
[ ] [ ]

Picosecond 4:
0 1 2 3 4 5 6
[S] [S] ... ... ( ) ... [ ]
[ ] [ ] [ ] [ ]
[ ] [S] [S]
[ ] [ ]

0 1 2 3 4 5 6
[ ] [ ] ... ... ( ) ... [ ]
[S] [S] [S] [S]
[ ] [ ] [ ]
[ ] [ ]

Picosecond 5:
0 1 2 3 4 5 6
[ ] [ ] ... ... [ ] (.) [ ]
[S] [S] [S] [S]
[ ] [ ] [ ]
[ ] [ ]

0 1 2 3 4 5 6
[ ] [S] ... ... [S] (.) [S]
[ ] [ ] [ ] [ ]
[S] [ ] [ ]
[ ] [ ]

Picosecond 6:
0 1 2 3 4 5 6
[ ] [S] ... ... [S] ... (S)
[ ] [ ] [ ] [ ]
[S] [ ] [ ]
[ ] [ ]

0 1 2 3 4 5 6
[ ] [ ] ... ... [ ] ... ( )
[S] [S] [S] [S]
[ ] [ ] [ ]
[ ] [ ]
</code></pre>

<p>In this situation, you are <em>caught</em> in layers <code>0</code> and <code>6</code>, because your packet entered the layer when its scanner was at the top when you entered it. You are <em>not</em> caught in layer <code>1</code>, since the scanner moved into the top of the layer once you were already there.</p>
<p>The <em>severity</em> of getting caught on a layer is equal to its <em>depth</em> multiplied by its <em>range</em>. (Ignore layers in which you do not get caught.) The severity of the whole trip is the sum of these values.  In the example above, the trip severity is <code>0*3 + 6*4 = <em>24</em></code>.</p>
<p>Given the details of the firewall you've recorded, if you leave immediately, <em>what is the severity of your whole trip</em>?</p>
</article>


In [25]:
with open("../input/day13.txt") as f:
    puzzle = f.read()

In [26]:
example = """
0: 3
1: 2
4: 4
6: 4
"""

In [48]:
def create_firewall(record: str) -> dict[int, int]:
    firewall = {}
    for line in record.strip().splitlines():
        d, r = map(int, line.split(": "))
        firewall[d] = r
    return firewall


def severity(record: str, start: int = 0) -> int:
    firewall = create_firewall(record)
    severity = 0
    for d, r in firewall.items():
        state, step = 0, 1
        for _ in range(d + start):
            state += step
            if state == r - 1 or state == 0:
                step *= -1

        if state == 0:
            severity += d * r

    return severity


print(f"Example: {severity(example)}")

Example: 24


In [49]:
print(f"Part I: {severity(puzzle)}")

Part I: 1476


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1476</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now, you need to pass through the firewall without being caught - easier said than done.</p>
<p>You can't control the <span title="Seriously, what network stack doesn't let you adjust the speed of light?">speed of the packet</span>, but you can <em>delay</em> it any number of picoseconds. For each picosecond you delay the packet before beginning your trip, all security scanners move one step. You're not in the firewall during this time; you don't enter layer <code>0</code> until you stop delaying the packet.</p>
<p>In the example above, if you delay <code>10</code> picoseconds (picoseconds <code>0</code> - <code>9</code>), you won't get caught:</p>
<pre><code>State after delaying:
 0   1   2   3   4   5   6
[ ] [S] ... ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]

Picosecond 10:
0 1 2 3 4 5 6
( ) [S] ... ... [ ] ... [ ]
[ ] [ ] [ ] [ ]
[S] [S] [S]
[ ] [ ]

0 1 2 3 4 5 6
( ) [ ] ... ... [ ] ... [ ]
[S] [S] [S] [S]
[ ] [ ] [ ]
[ ] [ ]

Picosecond 11:
0 1 2 3 4 5 6
[ ] ( ) ... ... [ ] ... [ ]
[S] [S] [S] [S]
[ ] [ ] [ ]
[ ] [ ]

0 1 2 3 4 5 6
[S] (S) ... ... [S] ... [S]
[ ] [ ] [ ] [ ]
[ ] [ ] [ ]
[ ] [ ]

Picosecond 12:
0 1 2 3 4 5 6
[S] [S] (.) ... [S] ... [S]
[ ] [ ] [ ] [ ]
[ ] [ ] [ ]
[ ] [ ]

0 1 2 3 4 5 6
[ ] [ ] (.) ... [ ] ... [ ]
[S] [S] [S] [S]
[ ] [ ] [ ]
[ ] [ ]

Picosecond 13:
0 1 2 3 4 5 6
[ ] [ ] ... (.) [ ] ... [ ]
[S] [S] [S] [S]
[ ] [ ] [ ]
[ ] [ ]

0 1 2 3 4 5 6
[ ] [S] ... (.) [ ] ... [ ]
[ ] [ ] [ ] [ ]
[S] [S] [S]
[ ] [ ]

Picosecond 14:
0 1 2 3 4 5 6
[ ] [S] ... ... ( ) ... [ ]
[ ] [ ] [ ] [ ]
[S] [S] [S]
[ ] [ ]

0 1 2 3 4 5 6
[ ] [ ] ... ... ( ) ... [ ]
[S] [S] [ ] [ ]
[ ] [ ] [ ]
[S] [S]

Picosecond 15:
0 1 2 3 4 5 6
[ ] [ ] ... ... [ ] (.) [ ]
[S] [S] [ ] [ ]
[ ] [ ] [ ]
[S] [S]

0 1 2 3 4 5 6
[S] [S] ... ... [ ] (.) [ ]
[ ] [ ] [ ] [ ]
[ ] [S] [S]
[ ] [ ]

Picosecond 16:
0 1 2 3 4 5 6
[S] [S] ... ... [ ] ... ( )
[ ] [ ] [ ] [ ]
[ ] [S] [S]
[ ] [ ]

0 1 2 3 4 5 6
[ ] [ ] ... ... [ ] ... ( )
[S] [S] [S] [S]
[ ] [ ] [ ]
[ ] [ ]
</code></pre>

<p>Because all smaller delays would get you caught, the fewest number of picoseconds you would need to delay to get through safely is <code>10</code>.</p>
<p><em>What is the fewest number of picoseconds</em> that you need to delay the packet to pass through the firewall without being caught?</p>
</article>

</main>


In [215]:
def state(d, r):
    if r == 3:
        r += 1
    elif r > 3:
        r = 4 + (r - 3) * 2
    return abs(d - r * round(d / r))


def passes(firewall: dict[int, list[int]], start: int = 0) -> bool:
    for d, r in firewall.items():
        if state(d + start, r) == 0:
            return False
    return True


def minimum_time_delay_not_caught(record: str) -> int:
    firewall = create_firewall(record)
    dt = 0
    while True:
        if passes(firewall, dt):
            return dt
        dt += 1


print(f"Example: {minimum_time_delay_not_caught(example)}")

Example: 10


In [216]:
print(f"Part II: {minimum_time_delay_not_caught(puzzle)}")

Part II: 3937334


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3937334</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>


<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
